# Quarter in-depth PI analysis

This report has been based on modified WANO PC approach. The purpose of this report is defining the units which need more support.

In [2]:
#install.packages('logging', repos='http://cran.us.r-project.org')

Analysing quarter

In [1]:
EndDate <- 201712

Read all the necessary data and function

In [2]:
source('shiny/data.r')

Criteria for FLR: the average two-year indicator is in the bottom decile in worlwide comparison

SP1, SP2 and SP5 - indicator is safety related.

Criteria for US7: 
- first filter: at least one scram occured during the last monitored quarter
- second filter: three or more scrams occured in the last 12 months

Criteria for FRI: calculated value is above the Defect Fuel Reference

In [3]:
#tail(r)
IDbyName <- function(name){return(subset(place,AbbrevLocName==name,LocId)[[1]])}

FLR: the average two-year indicator is in the bottom decile in worlwide comparison

In [4]:
flr_lim <- quantile(subset(r, IndicatorCode == 'FLR  ' & PeriodEndYrMn == EndDate & NumOfMonths == 24 & NonQualCode == ' ',ResultsValue)[[1]],probs=0.9)[[1]]
flr <- subset(r, IndicatorCode == 'FLR  ' & PeriodEndYrMn == EndDate & NumOfMonths == 24 & NonQualCode == ' ' & ResultsValue>=flr_lim,LocId)[[1]]
flr <- nameByID(flr)

Units which is above the safety system criteria (0.02 for SP1-2, 0.025 for SP5) for 3-yrs data window

In [5]:
sp12 <- nameByID(subset(r, IndicatorCode %in% c('SP1  ','SP2  ') & PeriodEndYrMn == EndDate & NumOfMonths == 36 & NonQualCode == ' ' & ResultsValue >=0.02,LocId)[[1]])
sp5 <- nameByID(subset(r, IndicatorCode == 'SP5  ' & PeriodEndYrMn == EndDate & NumOfMonths == 36 & NonQualCode == ' ' & ResultsValue >=0.025,LocId)[[1]])

Units which has at least one scram occured during the last monitored quarter

In [6]:
#tail(data)
scram <-nameByID(unique(subset(data,YrMn %in% c(EndDate-2, EndDate-1 ,EndDate) & ElementCode %in% c('C1   ','C3   ') & ElementValue>0, SourceId))[[1]])

Units which has three or more scrams for the last 12 months

In [7]:
scrams3 <- nameByID(unique(subset(data,YrMn > (EndDate-100) & ElementCode %in% c('C1   ','C3   ') & ElementValue>=3, SourceId))[[1]])

FRI: calculated value is above the Defect Fuel Reference

In [8]:
u <- uByType()
fri2 <- nameByID(subset(r, IndicatorCode == 'FRI  ' & PeriodEndYrMn == EndDate & NumOfMonths == 3 & NonQualCode == ' ' & ResultsValue >=300 & LocId %in% u$rType[[2]],LocId)[[1]])
fri45 <- nameByID(subset(r, IndicatorCode == 'FRI  ' & PeriodEndYrMn == EndDate & NumOfMonths == 3 & NonQualCode == ' ' & ResultsValue >=5e-4 & LocId %in% c(u$rType[[4]],u$rType[[5]]),LocId)[[1]])
fri1 <- nameByID(subset(r, IndicatorCode == 'FRI  ' & PeriodEndYrMn == EndDate & NumOfMonths == 3 & NonQualCode == ' ' & ResultsValue >=1e-3 & LocId %in% u$rType[[1]],LocId)[[1]])
fri3 <- nameByID(subset(r, IndicatorCode == 'FRI  ' & PeriodEndYrMn == EndDate & NumOfMonths == 3 & NonQualCode == ' ' & ResultsValue >=1e-3 & LocId %in% u$rType[[3]],LocId)[[1]])
fri <- c(fri1,fri2,fri3,fri45)

*Note: FRI criteria for AGR and LWCGR are based on Index calculation*

The whole list of units which have some issues

In [9]:
onFocus <- sort(unique(c(flr,sp12,sp5,scram,scrams3,fri)))

In [10]:
df <- data.frame(onFocus)
flrCol <- c()
sspiCol <-c()
S <- c()
S3 <- c()
FRI <- c()
Idx <- c()

for (i in df$onFocus){
    if (i %in% flr) flrCol <- c(flrCol,'X') else flrCol <- c(flrCol,'')
    if (i %in% c(sp12,sp5)) sspiCol <- c(sspiCol,'X') else sspiCol <- c(sspiCol,'')
    if (i %in% scram) S <- c(S,'X') else S <- c(S,'')
    if (i %in% scrams3) S3 <- c(S3,'X') else S3 <- c(S3,'')
    if (i %in% fri) FRI <- c(FRI,'X') else FRI <- c(FRI,'')
    index <- round(subset(idx,IndexId==4 & LocId==IDbyName(i) & PeriodEndYrMn==EndDate & NonQualCode==' ',IndexValue)[[1]])
    #print(paste(i,index))
    if (length(index) && is.numeric(index)) Idx <- c(Idx,index)
    else Idx <- c(Idx,0)
}
#Idx
#length(Idx)
df$FLR <- flrCol
df$SSPI <- sspiCol  
df$Scram <- S
df$Scrams <- S3
df$FRI <- FRI
df$Idx <- Idx
df
write.csv(df,'onFocus.csv')
length(df[,1])

onFocus,FLR,SSPI,Scram,Scrams,FRI,Idx
ANO,,X,,,,0
ANO Unit 2,X,,,,,66
Atucha 1,,,,X,,58
Atucha 2,X,,,,X,60
Beaver Valley Unit 1,,,X,,,99
Belleville 1,,,X,,,60
Borssele 1,,,X,,,78
Bruce 3,,,,,X,72
Bruce 7,,X,,,X,69
Brunswick Unit 1,,,,,X,92


[1] 98

The list of units which has *more than one* issue listed above - candidates for *units need more attention*

In [11]:
df2 <- data.frame()
for (n in 1:length(df[,1])){
    if (length(which(df[n,]=='X'))>=2)
        df2 <- rbind(df2,df[n,])
        }
df2
write.csv(df2,'onFocusShort.csv')
length(df2[,1])

,onFocus,FLR,SSPI,Scram,Scrams,FRI,Idx
4,Atucha 2,X,,,,X,60
9,Bruce 7,,X,,,X,69
15,Chasnupp 3,,,X,X,,74
16,Chasnupp 4,,,X,X,,0
23,Comanche Peak Unit 2,X,,X,,,70
28,Dukovany 2,X,X,,,,60
30,Dungeness B 3,X,,X,,X,64
31,Dungeness B 4,X,,X,,X,70
32,Fermi Unit 2,,X,,,X,79
36,Forsmark 3,X,,,,X,65


[1] 24

The list of units which has *more than two* issues listed above - candidates for *units on focus*

In [12]:
df3 <- data.frame()
for (n in 1:length(df[,1])){
    if (length(which(df[n,]=='X'))>=3)
        df3 <- rbind(df3,df[n,])
        }
df3
write.csv(df3,'onFocusVeryShort.csv')
length(df3[,1])

,onFocus,FLR,SSPI,Scram,Scrams,FRI,Idx
30,Dungeness B 3,X,,X,,X,64
31,Dungeness B 4,X,,X,,X,70
53,Kudankulam 1,X,X,X,,,62
73,Rajasthan 2,X,,,X,X,50


[1] 4

TODOs:
- check and print the related comments (if any);
- search the related events in the OE DB, sorting them by severity; using the Machine Learning approach will be useful;
- provide reccomendations for PR team
- add some text summary
- remove code

Conclusions:
- the existing PI Indexing system is slightly out-of-date and can be limitedly used even for initial investigation; The Index 4 needs some improvement;
- the existing WANO PC approach in selecting 'plants which need more attention' might be used for RC level, but for the whole WANO it represents about a quarter out of all units/plants which does not reflect the real picture;
- for WANO level the list of 'units which has more than one safety and reliability issues' might be used to define unit needs more attention;
- for WANO level the list of 'units which has more than two safety and reliability issues' might be used to define unit to be on focus;
- some units which have not good chemistry and industrial safety were not included into the lists above - probably the assesment criteria should be reevaluated;
- the last two tables with the assesment criteria might be included into the PI Quarter Report